In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install missingpy

In [ ]:
pip install scikit-learn==0.23.1

In [ ]:
# packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import imblearn
import warnings 
warnings.filterwarnings("ignore")
# impute missing value
from missingpy import MissForest

# SMOTE
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where
from imblearn.over_sampling import SMOTE

#Xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from xgboost.sklearn import XGBClassifier


#randomforest
from sklearn import ensemble

#logistic regression
from sklearn.linear_model import LogisticRegression


In [ ]:
#impute missing value
from missingpy import MissForest
imputer = MissForest(random_state=123)
data = pd.read_csv('../input/strokedata1/healthcare-dataset-stroke-data(1).csv')
data_=data.iloc[:,1:]
data_impute=imputer.fit_transform(data_)
data_impute=pd.DataFrame(data_impute)
data_impute.columns=data_.columns
data_impute.isnull().sum()

In [ ]:
#data preprocessing
oversample = SMOTE(random_state=123)
X, y = oversample.fit_resample(data_impute.iloc[:,:-1], data_impute.iloc[:,-1])

stroke_smote=pd.concat([X,y],axis=1)
stroke_smote.columns=data_impute.columns

for i in range(stroke_smote.shape[0]):
    if stroke_smote.iloc[i,0]>0.5:
        stroke_smote.iloc[i,0]=1
    else:
        stroke_smote.iloc[i,0]=0
        
for i in range(stroke_smote.shape[0]):
      stroke_smote.iloc[i,1]=int( stroke_smote.iloc[i,1])

for i in range(stroke_smote.shape[0]):
    if stroke_smote.iloc[i,2]>0.5:
        stroke_smote.iloc[i,2]=1
    else:
        stroke_smote.iloc[i,2]=0
        
for i in range(stroke_smote.shape[0]):
    if stroke_smote.iloc[i,3]>0.5:
        stroke_smote.iloc[i,3]=1
    else:
        stroke_smote.iloc[i,3]=0
        
for i in range(stroke_smote.shape[0]):
    if stroke_smote.iloc[i,4]>0.5:
        stroke_smote.iloc[i,4]=1
    else:
        stroke_smote.iloc[i,4]=0
        
for i in range(stroke_smote.shape[0]):
    if stroke_smote.iloc[i,5]>3.5:
        stroke_smote.iloc[i,5]=4
    elif stroke_smote.iloc[i,5]<3.5 and stroke_smote.iloc[i,5]>2.5 :
        stroke_smote.iloc[i,5]=3
    elif stroke_smote.iloc[i,5]<2.5 and stroke_smote.iloc[i,5]>1.5:
        stroke_smote.iloc[i,5]=2
    elif stroke_smote.iloc[i,5]<1.5 and stroke_smote.iloc[i,5]>0.5:
        stroke_smote.iloc[i,5]=1
    elif stroke_smote.iloc[i,5]<0.5:
        stroke_smote.iloc[i,5]=0

for i in range(stroke_smote.shape[0]):
    if stroke_smote.iloc[i,6]>0.5:
        stroke_smote.iloc[i,6]=1
    else:
        stroke_smote.iloc[i,6]=0

for i in range(stroke_smote.shape[0]):
    if stroke_smote.iloc[i,9]>3.5:
        stroke_smote.iloc[i,9]=4
    elif stroke_smote.iloc[i,9]<3.5 and stroke_smote.iloc[i,9]>2.5 :
        stroke_smote.iloc[i,9]=3
    elif stroke_smote.iloc[i,9]<2.5 and stroke_smote.iloc[i,9]>1.5:
        stroke_smote.iloc[i,9]=2
    elif stroke_smote.iloc[i,9]<1.5 and stroke_smote.iloc[i,9]>0.5:
        stroke_smote.iloc[i,9]=1
    elif stroke_smote.iloc[i,9]<0.5:
        stroke_smote.iloc[i,9]=0


In [ ]:
# xgboost
X= stroke_smote.iloc[:,:-1]
Y= stroke_smote.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=123)

clf = XGBClassifier(

    n_estimators=300,
    learning_rate= 0.03,
    max_depth=7,
    subsample=0.9,

    )
clf.fit(X_train,y_train,eval_set = [(X_test, y_test)],early_stopping_rounds=10,eval_metric='error',verbose=0)
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(' XGB accuracy = ',round(accuracy_score(y_test,y_pred),4)*100,'%')
print(' XGB f1_score = ',round(f1_score(y_test,y_pred),4)*100,'%')

In [ ]:
#Random forest
forest = ensemble.RandomForestClassifier(n_estimators=400,max_depth=7,min_samples_split=7,min_samples_leaf=4)
forest_fit = forest.fit(X_train, y_train)
# Prediction
y_pred = forest.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(' RF accuracy = ',round(accuracy_score(y_test,y_pred),4)*100,'%')
print(' RF f1_score = ',round(f1_score(y_test,y_pred),4)*100,'%')


In [ ]:
#Logistic regression
clf = LogisticRegression(random_state=0).fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(' logitstic = ',round(accuracy_score(y_test,y_pred),4)*100,'%')
print(' logitstic cf1_score = ',round(f1_score(y_test,y_pred),4)*100,'%')
